In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import requests
import json
import xarray as xr
import scipy.stats as stats
from scipy.stats import mannwhitneyu
from statsmodels.stats import multitest


# Assemble data inputs

## Reference genomes

All reference genomes in this project were derived from the Scientific Data publication [Single cell genomes of Prochlorococcus, Synechococcus, and sympatric microbes from diverse marine environments](https://www.nature.com/articles/sdata2018154) (Berube et al., 2018). The genome sequences and annotations used in this publication are hosted by JGI on the [IMG/ProPortal](https://img.jgi.doe.gov/cgi-bin/proportal/main.cgi) and can be downloaded using the [IMG Genome Cart tool](https://img.jgi.doe.gov/cgi-bin/mer/main.cgi?section=GenomeCart&page=genomeCart). However, there is currently a mis-match in some of the genome ids used on JGI, and those used to generate CyCOGs v6. 
- Working solution: use the genomes given to us by the Chisholm/Berube group. Later these will be uploaded to a public repository.
- Download genomes into the `../../data/genomes` directory.

## CyCOGs & genome metadata

Download the following files from the [figshare site associated with Berube et al. (2018)](https://doi.org/10.6084/m9.figshare.c.4037048.v1) to the `../../data/cycogs/` directory:
- File 4: [cycogs-genomes.tsv](https://figshare.com/articles/dataset/File_4_CyCOG_taxa/6007166)
- File 5: [cycogs.tsv](https://figshare.com/articles/dataset/File_5_CyCOG_definitions/6007169)
- File 14: [genome_assembly_summary_20180718.tsv](https://figshare.com/articles/dataset/File_14_Genome_assembly_summary/6281519)

For convenience, also add the derived metadata files `ortholog-metadata.csv` and `genome-metadata.csv` to the `../../data/cycogs/` directory.
- These files were generated by Stephen from various combined sources
- In the future they will be replaced with the original source files


In [2]:
# genome metadata

# filepaths
filepath_genome_metadata = '../../data/cycogs/genome-metadata.csv'
filepath_ortholog_metadata = '../../data/cycogs/ortholog-metadata.csv'

# ortholog metadata
ortho_df = pd.read_csv(filepath_ortholog_metadata)

# genome metadata
assembly_sum_df = pd.read_csv('../../data/cycogs/cycogsgenomes.tsv', sep='\t')
genome_df = pd.merge(
    left=assembly_sum_df[['IID', 'GROUP', 'IMG_ID', 'Completeness']], 
    right=pd.read_csv(filepath_genome_metadata).drop(columns='Completeness'), 
    left_on='IMG_ID', 
    right_on='BerubeProportalID',
    how='left'
)

# fix typos
genome_df.loc[genome_df['Clade'] == '5.1A-1', 'Clade'] = '5.1A-I'
genome_df.loc[genome_df['Clade'] == 'CDR2', 'Clade'] = 'CRD2'
genome_df.loc[genome_df['GROUP'] == 'Uncultured-marine-virus', 'GROUP'] = 'Virus'

genome_df


,IID,GROUP,IMG_ID,Completeness,BerubeProportalID,UpdatedProportalID,GenomeName,Genus,Ecotype,Clade,ReferenceType,IsolationLocation,Ecosystem,Latitude,Longitude,Depth(m),GenomeSize(bp),GeneCount
0,AG-311-D23,Prochlorococcus,2716884681,72.96,2.716885e+09,2.716885e+09,AG-311-D23,Prochlorococcus,Low light adapted (LL),LLI,SAG,South Pacific Ocean,Pelagic,-20.08,-70.8,20.0,1466304.0,1796.0
1,AG-311-I02,Prochlorococcus,2716884682,11.16,2.716885e+09,2.716885e+09,AG-311-I02,Prochlorococcus,Low light adapted (LL),LLI,SAG,South Pacific Ocean,Pelagic,-20.08,-70.8,20.0,195290.0,271.0
2,AG-311-I09,Prochlorococcus,2716884683,47.41,2.716885e+09,2.716885e+09,AG-311-I09,Prochlorococcus,High light adapted (HL),HLI,SAG,South Pacific Ocean,Pelagic,-20.08,-70.8,20.0,697970.0,812.0
3,AG-311-J05,Prochlorococcus,2716884684,34.64,2.716885e+09,2.716885e+09,AG-311-J05,Prochlorococcus,High light adapted (HL),HLI,SAG,South Pacific Ocean,Pelagic,-20.08,-70.8,20.0,623148.0,755.0
4,AG-311-J23,Prochlorococcus,2716884685,77.90,2.716885e+09,2.716885e+09,AG-311-J23,Prochlorococcus,Low light adapted (LL),LLI,SAG,South Pacific Ocean,Pelagic,-20.08,-70.8,20.0,1427538.0,1677.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,S-ShM2,Virus,651703106,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
766,Syn19,Virus,651703107,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767,Syn5,Virus,641201056,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
768,metaG-MbCM1,Virus,2595698410,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# add genome id and clade info into ortholog df

ortho_df['GenomeID'] = ortho_df['GenomeName'].map(genome_df.set_index('IID')['IMG_ID'])
ortho_df['Group'] = ortho_df['GenomeName'].map(genome_df.set_index('IID')['GROUP'])
ortho_df['Clade'] = ortho_df['GenomeName'].map(genome_df.set_index('IID')['Clade'])

# fill N/A clades with 'Not Assigned'
ortho_df['Clade'] = ortho_df['Clade'].fillna('Not Assigned')

ortho_df


,MappingName,OrthologID,GenomeName,GeneID,Annotation,GenomeID,Group,Clade
0,WH8102_2607658325,60000001,WH8102,2607658325,membrane protease FtsH catalytic subunit,2606217514,Synechococcus,5.1A-III
1,MIT0917_2681971350,60000001,MIT0917,2681971350,membrane protease FtsH catalytic subunit,2681812859,Prochlorococcus,LLI
2,AG-424-P18_2717338506,60000001,AG-424-P18,2717338506,membrane protease FtsH catalytic subunit,2716884419,Prochlorococcus,HLII
3,scB245a_521A19_2655604637,60000001,scB245a_521A19,2655604637,membrane protease FtsH catalytic subunit,2654587735,Prochlorococcus,Not Assigned
4,GFB01_2638208352,60000001,GFB01,2638208352,membrane protease FtsH catalytic subunit,2636415834,Synechococcus,Not Assigned
...,...,...,...,...,...,...,...,...
964917,AG-363-C02_2667889608,60040295,AG-363-C02,2667889608,hypothetical protein,2667527365,Prochlorococcus,LLII/III
964918,AG-363-C02_2667889615,60040295,AG-363-C02,2667889615,hypothetical protein,2667527365,Prochlorococcus,LLII/III
964919,AG-363-C02_2667890048,60040295,AG-363-C02,2667890048,hypothetical protein,2667527365,Prochlorococcus,LLII/III
964920,AG-363-C02_2667890054,60040295,AG-363-C02,2667890054,hypothetical protein,2667527365,Prochlorococcus,LLII/III


In [4]:
# start building ko mapping file

ko_map_df = ortho_df.groupby('OrthologID').GeneID.count().reset_index().rename(columns={'GeneID': 'TotalRefs'})
# add in genus and clade counts
for var in ['Group', 'Clade']:
    ko_map_df = ko_map_df.join(
        pd.DataFrame(ortho_df.groupby('OrthologID')[var].value_counts()).rename(
            columns={var: 'count'}).reset_index().pivot(
            columns=var, index='OrthologID', values='count').fillna(0), 
        on='OrthologID', 
        how='left'
    )
# convert dataframe back to ints
ko_map_df = ko_map_df.astype(int)
# add back CyCOG annotation
ko_map_df['DescriptionCyCOG'] = ko_map_df['OrthologID'].map(
    ortho_df[['OrthologID', 'Annotation']].drop_duplicates().set_index('OrthologID')['Annotation']
)

ko_map_df


,OrthologID,TotalRefs,Prochlorococcus,Synechococcus,Virus,5.1-UC-A,5.1A-I,5.1A-II,5.1A-III,5.1A-III/XV,...,CRD2,HLI,HLII,HLIII,HLVI,LLI,LLII/III,LLIV,Not Assigned,DescriptionCyCOG
0,60000001,1376,1218,158,0,3,13,17,5,3,...,17,195,440,1,20,172,79,66,266,membrane protease FtsH catalytic subunit
1,60000002,1453,1296,157,0,3,12,18,6,3,...,10,196,464,0,25,184,85,66,299,ATP-dependent Clp protease ATP-binding subunit...
2,60000003,1387,1231,156,0,3,8,16,6,2,...,12,203,446,1,21,180,79,65,259,ATP-dependent Clp protease proteolytic subunit...
3,60000004,1631,1376,255,0,7,20,23,11,12,...,22,170,477,0,31,252,97,127,259,hypothetical protein
4,60000005,990,882,108,0,2,9,11,3,3,...,8,126,323,0,19,123,62,46,200,chaperonin GroEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40290,60040291,4,4,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,0,hypothetical protein
40291,60040292,4,4,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,0,Tryptophan-rich Synechocystis species C-termin...
40292,60040293,5,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,Putative transposase
40293,60040294,6,0,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,sulfate transport system substrate-binding pro...


In [5]:
# check that there is a directory for each genome

data_path = Path('../../data/genomes')

count = 0
for g_id in ortho_df['GenomeID'].unique():
    if not os.path.isdir(data_path / str(g_id)):
        print(f'{g_id} directory not found: {data_path / str(g_id)}')
        count += 1
        
print(f'{count} total missing directories')


0 total missing directories


In [6]:
# import all kegg annotations as a df

ko_df = pd.DataFrame()
for g_id in ortho_df['GenomeID'].unique():
    df = pd.read_csv(data_path / f'{g_id}/{g_id}.ko.tab.txt', sep='\t')
    if len(ko_df) == 0:
        ko_df = df
    else:
        ko_df = pd.concat([ko_df, df])
        
ko_df = ko_df.reset_index(drop=True)

ko_df


,gene_oid,gene_length,percent_identity,query_start,query_end,subj_start,subj_end,evalue,bit_score,ko_id,ko_name,EC,img_ko_flag
0,2607658051,465,99.8,1,465,1,465,0.000000e+00,906.4,KO:K02313,chromosomal replication initiator protein,NaN,Yes
1,2607658053,410,100.0,1,410,11,420,0.000000e+00,856.7,KO:K00799,glutathione S-transferase [EC:2.5.1.18],EC:2.5.1.18,Yes
2,2607658055,455,100.0,1,455,1,455,0.000000e+00,895.6,KO:K00383,glutathione reductase (NADPH) [EC:1.8.1.7],EC:1.8.1.7,Yes
3,2607658058,198,100.0,1,198,1,198,0.000000e+00,409.1,KO:K02276,cytochrome c oxidase subunit III [EC:1.9.3.1],EC:1.9.3.1,Yes
4,2607658059,562,100.0,1,562,1,562,0.000000e+00,1173.7,KO:K02274,cytochrome c oxidase subunit I [EC:1.9.3.1],EC:1.9.3.1,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492987,2610145142,243,35.7,19,239,48,257,4.300000e-27,123.6,KO:K03791,putative chitinase,NaN,Yes
492988,2610145144,309,37.5,4,292,17,311,0.000000e+00,198.4,KO:K03086,RNA polymerase primary sigma factor,NaN,Yes
492989,2610145149,109,41.2,1,95,1,97,1.500000e-13,77.4,KO:K03111,single-strand DNA-binding protein,NaN,Yes
492990,2610145152,261,40.5,5,254,10,269,2.000000e-43,177.9,KO:K10906,exodeoxyribonuclease VIII [EC:3.1.11.-],EC:3.1.11.-,Yes


# Deduplicate gene annotations

**Problem:** Some genes (specific nucleotide sequence from particular reference genome) have more than one KO annotation listed. 
- Based on the counts below, this comes out to 16,400 (15038 + 1352 + 9 + 1) out of 301,499 annotated genes, or ~5.44%. 
- Most of these are annotations that actually have the same KO number, but just a different EC number or something. Only 2,207 genes (0.73%) have multiple distinct KO annotations
- Of these, qualitatively it seems like the annotations are usually quite similar to one another (e.g. same pathway, or different subunits of the same protein complex)

**Proposed solution:** Since the KO annotations are based on an HMM search, first select the annotation with the lowest E-value. Then if the E-values are exactly the same, randomly select one annotaiton or the other


In [7]:
# how many genes have more than one ko number assigned?

print(ko_df['gene_oid'].value_counts().value_counts())


1    440542
2     23227
3      1973
4        15
6         2
5         1
Name: gene_oid, dtype: int64


In [8]:
# deeper look: examine genes with more than one annotation

counts = ko_df['gene_oid'].value_counts()
ko_counts = ko_df[ko_df['gene_oid'].isin(counts[counts.gt(1)].index)].groupby('gene_oid')['ko_id'].nunique()
print(ko_counts.value_counts())    # most have the same ko id, just different EC numbers or something

ko_df[ko_df['gene_oid'].isin(ko_counts[ko_counts.gt(1)].index)]


1    21836
2     3333
3       48
4        1
Name: ko_id, dtype: int64


,gene_oid,gene_length,percent_identity,query_start,query_end,subj_start,subj_end,evalue,bit_score,ko_id,ko_name,EC,img_ko_flag
1549,2681971726,240,93.75,1,240,1,240,0.000000e+00,513.0,KO:K06182,23S rRNA pseudouridine2604 synthase [EC:5.4.99...,EC:5.4.99.21,Yes
1550,2681971726,240,93.75,1,240,1,240,0.000000e+00,513.0,KO:K06183,16S rRNA pseudouridine516 synthase [EC:5.4.99.19],EC:5.4.99.19,Yes
1561,2681971798,467,89.08,1,467,1,467,0.000000e+00,960.0,KO:K04094,methylenetetrahydrofolate--tRNA-(uracil-5-)-me...,EC:2.1.1.74,Yes
1562,2681971798,467,90.15,1,467,1,467,0.000000e+00,969.0,KO:K03495,tRNA uridine 5-carboxymethylaminomethyl modifi...,NaN,Yes
1620,2681971886,348,80.77,1,338,1,338,0.000000e+00,680.0,KO:K08919,chlorophyll a/b binding light-harvesting prote...,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492225,2717343036,149,41.86,15,140,4,132,6.500000e-31,126.0,KO:K00980,glycerol-3-phosphate cytidylyltransferase [EC:...,EC:2.7.7.39,Yes
492226,2717343036,149,44.78,15,145,4,137,3.000000e-36,141.0,KO:K03272,D-beta-D-heptose 7-phosphate kinase / D-beta-D...,EC:2.7.1.167,Yes
492227,2717343036,149,44.78,15,145,4,137,3.000000e-36,141.0,KO:K03272,D-beta-D-heptose 7-phosphate kinase / D-beta-D...,EC:2.7.7.70,Yes
492269,2717343185,112,100.00,1,112,1,112,1.400000e-73,243.0,KO:K04751,nitrogen regulatory protein P-II 1,NaN,Yes


In [9]:
# deduplicate genes with more than one annotation

# for each gene_oid's set of annotations, select the one with the lowest e-value
# this step will also randomly select one of the annotations to propogate in cases with the same KO 
# but different EC numbers, or different KOs but same e-value

ko_df = ko_df.loc[ko_df.groupby('gene_oid')['evalue'].idxmin(), :].reset_index(drop=True)
ko_df


,gene_oid,gene_length,percent_identity,query_start,query_end,subj_start,subj_end,evalue,bit_score,ko_id,ko_name,EC,img_ko_flag
0,638291477,531,48.40,4,522,3,549,0.000000e+00,461.5,KO:K17680,twinkle protein [EC:3.6.4.12],EC:3.6.4.12,Yes
1,638291481,243,43.10,5,240,3,242,1.500000e-37,158.3,KO:K02335,DNA polymerase I [EC:2.7.7.7],EC:2.7.7.7,Yes
2,638291512,235,61.80,26,231,1,207,0.000000e+00,253.1,KO:K03465,thymidylate synthase (FAD) [EC:2.1.1.148],EC:2.1.1.148,Yes
3,638310966,295,49.20,2,294,32,309,0.000000e+00,283.1,KO:K06223,DNA adenine methylase [EC:2.1.1.72],EC:2.1.1.72,Yes
4,638311025,200,32.50,49,200,48,186,1.100000e-12,75.5,KO:K07336,PKHD-type hydroxylase [EC:1.14.11.-],EC:1.14.11.-,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
465755,2721491637,397,65.72,27,379,39,383,0.000000e+00,514.0,KO:K02010,iron(III) transport system ATP-binding protein...,EC:3.6.3.30,Yes
465756,2721491638,266,63.67,10,264,5,250,3.500000e-107,354.0,KO:K01069,hydroxyacylglutathione hydrolase [EC:3.1.2.6],EC:3.1.2.6,Yes
465757,2721491639,219,87.26,2,213,1,212,9.000000e-129,411.0,KO:K00765,ATP phosphoribosyltransferase [EC:2.4.2.17],EC:2.4.2.17,Yes
465758,2721491640,599,81.64,1,599,1,599,0.000000e+00,1060.0,KO:K06147,"ATP-binding cassette, subfamily B, bacterial",NaN,Yes


# Deduplicate CyCOG annotations

**Problem:** Some CyCOG (Clusters of Orthologous Genes) consist of genes with discordant KO annotations
- Out of 20,502 CyCOGs represented by the clades we're considering in this study, 18,132 had no KO annotation, meaning just 2,370 (11.56%) contained at least one gene with a KO annotation.
- In most of the annotated CyCOGs, all gene members had a concordant KO annotation. In 93 (3.94% of annotated CyCOGs), the KO annotations were discordant among gene members.
- Of these, qualitatively it seems like the discordant annotations are usually quite similar to one another (e.g. same pathway, or different subunits of the same protein complex)

**Proposed solution:** For each CyCOG, select the annotation applied to the majority of members as the representatitve CyCOG annotation. 


In [10]:
# join kegg annotations onto reference gene set

annot_df = pd.merge(ortho_df, ko_df, left_on='GeneID', right_on='gene_oid', how='left')

# how many unique ko annotations per ortholog group?
print(annot_df.groupby('OrthologID').ko_id.nunique().value_counts())

# drop the sequences without a ko annotation
annot_df = annot_df[annot_df['ko_id'].notna()]

annot_df


0     36693
1      3474
2        94
3        14
4        13
7         2
9         2
5         1
8         1
10        1
Name: ko_id, dtype: int64


,MappingName,OrthologID,GenomeName,GeneID,Annotation,GenomeID,Group,Clade,gene_oid,gene_length,...,query_start,query_end,subj_start,subj_end,evalue,bit_score,ko_id,ko_name,EC,img_ko_flag
0,WH8102_2607658325,60000001,WH8102,2607658325,membrane protease FtsH catalytic subunit,2606217514,Synechococcus,5.1A-III,2.607658e+09,637.0,...,1.0,637.0,1.0,637.0,0.0,1240.7,KO:K03798,cell division protease FtsH [EC:3.4.24.-],EC:3.4.24.-,Yes
1,MIT0917_2681971350,60000001,MIT0917,2681971350,membrane protease FtsH catalytic subunit,2681812859,Prochlorococcus,LLI,2.681971e+09,640.0,...,1.0,640.0,1.0,640.0,0.0,1370.0,KO:K03798,cell division protease FtsH [EC:3.4.24.-],EC:3.4.24.-,Yes
2,AG-424-P18_2717338506,60000001,AG-424-P18,2717338506,membrane protease FtsH catalytic subunit,2716884419,Prochlorococcus,HLII,2.717339e+09,584.0,...,1.0,584.0,1.0,584.0,0.0,1260.0,KO:K03798,cell division protease FtsH [EC:3.4.24.-],EC:3.4.24.-,Yes
3,scB245a_521A19_2655604637,60000001,scB245a_521A19,2655604637,membrane protease FtsH catalytic subunit,2654587735,Prochlorococcus,Not Assigned,2.655605e+09,584.0,...,1.0,584.0,1.0,584.0,0.0,1250.0,KO:K03798,cell division protease FtsH [EC:3.4.24.-],EC:3.4.24.-,Yes
4,GFB01_2638208352,60000001,GFB01,2638208352,membrane protease FtsH catalytic subunit,2636415834,Synechococcus,Not Assigned,2.638208e+09,646.0,...,1.0,643.0,1.0,643.0,0.0,1134.4,KO:K03798,cell division protease FtsH [EC:3.4.24.-],EC:3.4.24.-,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964878,KORDI-49_2507312281,60040286,KORDI-49,2507312281,hypothetical protein,2507262011,Synechococcus,5.1A-WPC1,2.507312e+09,1889.0,...,1.0,1889.0,1.0,1889.0,0.0,3815.0,KO:K07004,NaN,NaN,Yes
964910,GFB01_2638207649,60040294,GFB01,2638207649,sulfate transport system substrate-binding pro...,2636415834,Synechococcus,Not Assigned,2.638208e+09,289.0,...,7.0,280.0,6.0,279.0,0.0,389.4,KO:K02048,sulfate transport system substrate-binding pro...,NaN,Yes
964911,GFB01_2638207687,60040294,GFB01,2638207687,sulfate transport system substrate-binding pro...,2636415834,Synechococcus,Not Assigned,2.638208e+09,338.0,...,7.0,338.0,6.0,338.0,0.0,449.5,KO:K02048,sulfate transport system substrate-binding pro...,NaN,Yes
964912,GFB01_2638207688,60040294,GFB01,2638207688,sulfate transport system substrate-binding pro...,2636415834,Synechococcus,Not Assigned,2.638208e+09,348.0,...,10.0,348.0,19.0,363.0,0.0,502.3,KO:K02048,sulfate transport system substrate-binding pro...,NaN,Yes


In [11]:
# look at non-unique ko mappings

counts = annot_df.groupby('OrthologID').ko_id.nunique()
collisions_df = annot_df[annot_df['OrthologID'].isin(counts[counts.gt(1)].index)].groupby('OrthologID')[
    ['ko_id', 'ko_name']].value_counts().reset_index()

pd.set_option('max_colwidth', None)
collisions_df


,OrthologID,ko_id,ko_name,0
0,60000002,KO:K03696,ATP-dependent Clp protease ATP-binding subunit ClpC,443
1,60000002,KO:K03695,ATP-dependent Clp protease ATP-binding subunit ClpB,417
2,60000006,KO:K11329,"two-component system, OmpR family, response regulator RpaB",452
3,60000006,KO:K07659,"two-component system, OmpR family, phosphate regulon response regulator OmpR",73
4,60000006,KO:K07657,"two-component system, OmpR family, phosphate regulon response regulator PhoB",34
...,...,...,...,...
317,60014461,KO:K05879,"dihydroxyacetone kinase, C-terminal domain [EC:2.7.1.-]",1
318,60014461,KO:K05878,"dihydroxyacetone kinase, N-terminal domain [EC:2.7.1.-]",1
319,60014461,KO:K00863,dihydroxyacetone kinase [EC:2.7.1.29],1
320,60040231,KO:K11951,bicarbonate transport system permease protein,1


In [12]:
# deduplicate the discordant ortholog-ko mapping by simple majority vote

ko_count_df = pd.DataFrame(annot_df.groupby('OrthologID').ko_id.value_counts()).rename(
    columns={'ko_id': 'count'}).reset_index()
ko_count_df = ko_count_df.loc[ko_count_df.groupby('OrthologID')['count'].idxmax(), :]

# add in KO annotation
name_map = annot_df[['ko_id', 'ko_name']].drop_duplicates()
name_map['DescriptionKO'] = name_map['ko_name'].str.split(' \[EC:').str[0]
name_map = name_map.loc[name_map['ko_id'].drop_duplicates().index]    # pick one description from duplicates
ko_count_df = pd.merge(left=ko_count_df, right=name_map[['ko_id', 'DescriptionKO']], on='ko_id', how='left')

# remove prefix from KOID
ko_count_df['KOID'] = ko_count_df['ko_id'].str[3:]
# rename count column and drop old ko_id column
ko_count_df = ko_count_df[['OrthologID', 'KOID', 'DescriptionKO', 'count']].rename(
    columns={'count': 'NRefsKO'}).set_index('OrthologID')
# calculate refs with KO mapping other than the one chosen
ko_count_df['NRefsOtherKO'] = annot_df.groupby('OrthologID').GeneID.count() - ko_count_df['NRefsKO']

# add ko_count_df into ko_map_df
ko_map_df = pd.merge(ko_map_df, ko_count_df, on='OrthologID', how='left')

pd.reset_option('max_colwidth')
ko_map_df


,OrthologID,TotalRefs,Prochlorococcus,Synechococcus,Virus,5.1-UC-A,5.1A-I,5.1A-II,5.1A-III,5.1A-III/XV,...,HLVI,LLI,LLII/III,LLIV,Not Assigned,DescriptionCyCOG,KOID,DescriptionKO,NRefsKO,NRefsOtherKO
0,60000001,1376,1218,158,0,3,13,17,5,3,...,20,172,79,66,266,membrane protease FtsH catalytic subunit,K03798,cell division protease FtsH,1291.0,0.0
1,60000002,1453,1296,157,0,3,12,18,6,3,...,25,184,85,66,299,ATP-dependent Clp protease ATP-binding subunit...,K03696,ATP-dependent Clp protease ATP-binding subunit...,443.0,417.0
2,60000003,1387,1231,156,0,3,8,16,6,2,...,21,180,79,65,259,ATP-dependent Clp protease proteolytic subunit...,K01358,"ATP-dependent Clp protease, protease subunit",1360.0,0.0
3,60000004,1631,1376,255,0,7,20,23,11,12,...,31,252,97,127,259,hypothetical protein,NaN,NaN,NaN,NaN
4,60000005,990,882,108,0,2,9,11,3,3,...,19,123,62,46,200,chaperonin GroEL,K04077,chaperonin GroEL,935.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40290,60040291,4,4,0,0,0,0,0,0,0,...,0,4,0,0,0,hypothetical protein,NaN,NaN,NaN,NaN
40291,60040292,4,4,0,0,0,0,0,0,0,...,0,4,0,0,0,Tryptophan-rich Synechocystis species C-termin...,NaN,NaN,NaN,NaN
40292,60040293,5,0,5,0,0,0,0,0,0,...,0,0,0,0,5,Putative transposase,NaN,NaN,NaN,NaN
40293,60040294,6,0,6,0,0,0,0,0,0,...,0,0,0,0,6,sulfate transport system substrate-binding pro...,K02048,sulfate transport system substrate-binding pro...,4.0,0.0


In [13]:
# there are many different CyCOG orthologies annotated with the same KO group -- look at some of these

ko_map_counts = ko_map_df['KOID'].value_counts()
print('There are {} of {} KOs with a non-unique CyCOGID-KOID mapping:'.format(
    len(ko_map_counts[ko_map_counts.gt(1)]), ko_map_df['KOID'].nunique()))
print(ko_map_counts[ko_map_counts.gt(1)].head(20))

# look at serralysin KOIDs
ko_map_df[ko_map_df['KOID'] == 'K01406'].head(10)

# NOTE: these are the CyCOGs in which the serralysin annotation is the majority KEGG annotation

There are 669 of 1748 KOs with a non-unique CyCOGID-KOID mapping:
K06147    62
K01784    41
K01953    37
K01154    28
K03090    24
K07257    23
K01652    20
K00067    18
K01790    18
K00615    18
K00059    17
K02500    17
K00604    17
K02501    16
K00058    16
K03086    16
K00161    15
K05577    15
K00558    14
K01710    14
Name: KOID, dtype: int64


,OrthologID,TotalRefs,Prochlorococcus,Synechococcus,Virus,5.1-UC-A,5.1A-I,5.1A-II,5.1A-III,5.1A-III/XV,...,HLVI,LLI,LLII/III,LLIV,Not Assigned,DescriptionCyCOG,KOID,DescriptionKO,NRefsKO,NRefsOtherKO
1829,60001830,488,322,166,0,14,4,7,8,26,...,0,23,0,149,107,hypothetical protein,K01406,serralysin,4.0,4.0
1882,60001883,507,495,12,0,0,0,0,2,0,...,1,384,0,3,36,protein of unknown function (DUF4214),K01406,serralysin,14.0,15.0
1887,60001888,351,313,33,5,2,0,1,2,7,...,9,44,17,119,61,hypothetical protein,K01406,serralysin,21.0,23.0
2364,60002365,127,96,31,0,1,1,5,0,1,...,0,25,1,2,19,hypothetical protein,K01406,serralysin,4.0,0.0
5973,60005974,7,0,7,0,0,0,2,0,0,...,0,0,0,0,0,serralysin,K01406,serralysin,6.0,0.0
7889,60007890,4,4,0,0,0,0,0,0,0,...,0,4,0,0,0,hypothetical protein,K01406,serralysin,1.0,0.0
12357,60012358,2,2,0,0,0,0,0,0,0,...,0,0,0,2,0,hypothetical protein,K01406,serralysin,1.0,0.0
16844,60016845,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,serralysin,K01406,serralysin,1.0,0.0
21516,60021517,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,Matrixin/Peptidase M10 serralysin C terminal,K01406,serralysin,1.0,0.0
21871,60021872,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,Peptidase M10 serralysin C terminal,K01406,serralysin,1.0,0.0


In [14]:
# for each "serralysin" CyCOG family, look at all of the KEGG annotations recruited

serralysin_cycogs = [60001830, 60001883, 60001888, 60002365]
serralysin_proteins_df = annot_df[annot_df['OrthologID'].isin(serralysin_cycogs)]
serralysin_proteins_df.groupby(['OrthologID'])[['ko_id', 'ko_name']].value_counts()

# review reference on type IV pili: https://www.nature.com/articles/s41579-019-0195-4


OrthologID  ko_id      ko_name                                                
60001830    KO:K01406  serralysin [EC:3.4.24.40]                                   4
            KO:K01113  alkaline phosphatase D [EC:3.1.3.1]                         2
            KO:K01317  acrosin [EC:3.4.21.10]                                      1
            KO:K03646  colicin import membrane protein                             1
60001883    KO:K01406  serralysin [EC:3.4.24.40]                                  14
            KO:K09691  lipopolysaccharide transport system ATP-binding protein     9
            KO:K03932  polyhydroxybutyrate depolymerase                            4
            KO:K12544  S-layer protein                                             2
60001888    KO:K01406  serralysin [EC:3.4.24.40]                                  21
            KO:K02650  type IV pilus assembly protein PilA                        20
            KO:K10924  MSHA pilin protein MshA                         

In [15]:
# look at all CyCOGs with any reference containing a "serralysin" annotation

annot_df[annot_df['ko_id'] == 'KO:K01406'].groupby('OrthologID').ko_id.value_counts()

# these are all the same CyCOGs as above, so no new CyCOG families with serralysin annotations


OrthologID  ko_id    
60001830    KO:K01406     4
60001883    KO:K01406    14
60001888    KO:K01406    21
60002365    KO:K01406     4
60005974    KO:K01406     6
60007890    KO:K01406     1
60012358    KO:K01406     1
60016845    KO:K01406     1
60021517    KO:K01406     1
60021872    KO:K01406     1
Name: ko_id, dtype: int64

# Get "serralysin" family strains

In [23]:
# get list of all strains with representatives in one of four "serralysin" CyCOG families

serralysin_cycogs = [60001830, 60001883, 60001888, 60002365]
serralysin_refs_df = ortho_df[ortho_df.OrthologID.isin(serralysin_cycogs)]
# add in isolate information
serralysin_refs_df['RefType'] = serralysin_refs_df['GenomeID'].map(
    assembly_sum_df.set_index('IMG_ID')['TYPE'])

# sort
serralysin_refs_df = serralysin_refs_df.sort_values(
    ['Group', 'Clade', 'RefType', 'GenomeName', 'OrthologID']
).reset_index(drop=True)

# save csv
serralysin_refs_df.to_csv('data/serralysin_cycog_references.csv', index=False)

# show how many copies per reference
pd.set_option('display.max_rows', 500)
serralysin_refs_df.groupby(['Group', 'Clade', 'RefType', 'GenomeName']).OrthologID.value_counts()


/var/folders/w0/7pz54fg92csc1bv996cgrss80000gn/T/ipykernel_40222/1532952077.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serralysin_refs_df['RefType'] = serralysin_refs_df['GenomeID'].map(


Group            Clade         RefType  GenomeName      OrthologID
Prochlorococcus  HLI           SAG      AG-321-G20      60002365       1
                                        AG-321-M21      60002365       4
                                        AG-321-P21      60002365       2
                                        AG-335-C21      60001883       1
                                                        60001888       1
                                        AG-335-E22      60002365       4
                                        AG-388-B05      60001888       2
                                        AG-402-N10      60001888       2
                                        AG-469-F22      60001888       3
                                                        60001830       2
                                        AG-670-L08      60001888       5
                                                        60001830       1
                                        AG-676-L21      6